In [101]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

## 지도학습 : 선형회귀 

수작업으로 그래프를 그리며 데이터셋에 적합시키는 모델은 시간이 무한정 소요되므로 적합하지 않다.  
더 복잡한 머신러닝/딥러닝 문제들로 들어가면 비선형 관계를 찾기 시작할 것이며  
변수가 많은 복잡한 식을 사용하게 될 것이다. 

여기에서 배우게 될 머신러닝 훈련 기법은 그 문제들에도 동일하게 적용할 수 있다. 

In [3]:
# csv 파일을 읽고 레코드를 출력한다. 
features = pd.read_csv('../data/house_price.csv')

In [4]:
#  처음 8개의 점을 훈련셋으로 분리(0~7)  
x_train = features[['Area', 'Locality']].values[:7]
y_train = features[['Price']].values[:7]

# 마지막 2개의 점을 테스트 셋으로 분할
x_test = features[['Area', 'Locality']].values[7:]
y_test = features[['Price']].values[7:]

훈련 데이터셋은 모델의 가중치 학습에 사용되고   
테스트 데이터셋은 모델이 처음 본 데이터에도 잘 예측하고 테스트하는 지 확인하는 데 사용될 것이다. 

최대한 잘 적합되었다는 것은 어떻게 판단할 수 있을까??  
이를 위해 비용함수가 필요하다.  
비용함수는 기본적으로 모델의 예측값이 실제 값과 얼마나 차이나는가를 측정하는 수단이다. 

비용 함수는 예측값과 실제 값의 차이를 계량해야 한다. 수치값 출력을 예측하는 경우,  
각 훈련 데이터의 예측값과 실제 값의 차이를 합산해 구할 수 있다.  
만일 클래스를 예측하는 경우라면, 분류 오차를 계량화하는 함수를 사용할 수 있을 것이다. 

비용 함수를 오차 함수라고도 하는데, 이는 오차함수가 예측 상의 오차를 계량화하기 때문이다.  
비용함수를 최적화의 목적함수로 사용할 수 있다. 가중치 값을 최적화해 비용 함수의 값을 최소화한다.  
이것은 전통적인 최적화 문제가 된다. 

### 경사하강 최적화 

모든 X들을 모델에 통과시켜 Y를 예측한다.  
예측값들을 실제 값들과 비교해 오차를 구한다.  
비용함수의 각 가중치 및 편향에 대한 경사도를 구한다. 

경사도는 기본적으로 각 가중치/편향에 대한 비용함수의 편도함수(partial derivative)이다. 

이 경사도는 특정한 가중치 및 편향이 비용에 미치는 영향의 크기와 방향을 알려준다.  
이 값을 이용해 비용을 감소시키는 방향으로 가중치와 편향을 보정한다 .

- MAE(평균 절댓값 오차) : mean(abs(y-y'))
- MSE(평균 제곱 오차) : mean((y-y')^2)

가중치를 경사도 방향으로 얼마만큼 보정할 것인지는 학습률이라는 상수 파라미터로 조절한다.  
학습률을 너무 크게 잡으면 최소값을 지나쳐서 곡선의 다른 쪽에 떨어지게 될 수 있다.  
학습률이 너무 작으면 가중치가 충분히 보정되지 않으므로 학습 과정이 너무 느려진다.  
일반적으로 0.05로 시작하는 것이 무난하다. 

In [5]:
# sklearn의 내장함수를 써서 선형 회귀 모델을 적용한다. 
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x_train, y_train)
print("Model weights are: ", model.coef_)
print("Model intercept is: ", model.intercept_)

# 테스트 데이터셋의 한 샘플에 대한 예측 
print('predicting for ', x_test[0])
print('Expected Value: ', y_test[0])
print("Predicted Value: ", model.predict([[95, 5]]))

Model weights are:  [[ 0.19319908 14.27643887]]
Model intercept is:  [-49.75481457]
predicting for  [225   4]
Expected Value:  [60]
Predicted Value:  [[39.98129196]]


회귀 모델의 평가에는 MSE/MAE 측도가 사용되며, 그 결과값이 높게 나온다면 다른 회귀 모델을 생각해봐야 한다. 

만일 선형 모델을 사용하는 데 오차 값이 계속 나온다면 일반적으로 더 복잡한 비선형 모델을 검토하기 시작해야 한다.  
비선형회귀 방법 중에 가장 유명한 것이 신경망이다. 

신경망으로 데이터 내의 비선형성을 포착할 수 있으며, 오차가 작은 모델을 찾을 수 있다.  
신경망처럼 복잡한 모델에 대해서 실제 값과 예측값 간의 오차를 네트워크로 전파하고  
가중치와 편향에 대한 비용함수의 경사(gradient)를 빠르게 계산할 수 있게 해주는 역전파라고 하는 매우 정교한 알고리즘을 볼 수 있다. 

## 지도학습 - 분류

분류에서 결과 또는 종속변수는 값이 아니라 클래스 멤버십이다.  
결과는 0에서부터 클래스 개수까지의 정수 값을 가질 수 있다.  

선형 회귀의 값을 선형 분류의 값으로 설정할 수 있다.  


이중 분류의 경우, 먼저 선형 회귀에서처럼 변수들 간의 선형 관계를 학습한다.  
그런 다음 sigmoid 활성화 함수를 서서 0과 1 사이의 값으로 변환한다. 

이 선형 가중 합계를 sigmoid 함수에 넣은 0과 1 사이의 값이  
어떤 임계값 이상이면 1로 분류하고, 아니면 0으로 분류한다. 

신경망을 이용하여 다중 클래스 분류 문제로 확장할 수 있다. 

주택의 면적/크기, 가격, 구입 여부를 나타내는 열 한 개가 추가된 데이터가 있다고 하자.  
이 열은 주택을 사면 1, 안 사면 0으로 표시될 것이다.  
이제 고객이 주택을 살 것인지 안 살 것인지 예측하는 이유에 대한 모델을 컴퓨터가 예측하게 하고 싶다.  
앞의 예제에서처럼, 데이터를 훈련셋과 테스트 셋으로 분리하자.  
마지막 두 점을 테스트용 데이터로 사용한다. 

In [10]:
features = pd.read_csv('../data/house_sale.csv')

In [11]:
features.head()

,Area,Locality,Price,Buy
0,100,4,30,0
1,250,5,80,1
2,220,5,80,1
3,105,6,40,1
4,150,8,100,0


In [13]:
# 처음 8개의 점을 훈련 데이터로 분리(0~7)
x_train = features[['Area', 'Locality', 'Price']].values[:8]
x_temp = features[['Area', 'Locality', 'Price']][:8]

In [14]:
print(x_train, type(x_train)) # numpy.ndarray
print(x_temp, type(x_temp)) # pandas.dataframe

[[100   4  30]
 [250   5  80]
 [220   5  80]
 [105   6  40]
 [150   8 100]
 [180   9 120]
 [225   4  60]
 [ 95   5  40]] <class 'numpy.ndarray'>
   Area  Locality  Price
0   100         4     30
1   250         5     80
2   220         5     80
3   105         6     40
4   150         8    100
5   180         9    120
6   225         4     60
7    95         5     40 <class 'pandas.core.frame.DataFrame'>


In [15]:
y_train = features['Buy'].values[:8]

In [16]:
# 마지막 2 점을 테스트 데이터로 분리
x_test = features[['Area', 'Locality', 'Price']].values[8:]
y_test = features['Buy'].values[8:]

이제 이 훈련 데이터에 로지스틱 회귀 모델을 적용하자. 그리고 훈련된 모델로  
두 테스트 점들에 대해 예측해보자. 

In [18]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(x_train, y_train)

# 테스트 데이터에서 예측
y_pred = model.predict(x_test)

# 예상값 출력 
print(y_pred)

# 마지막 2 점들은 테스트 데이터로 
print(y_test)

[1 0]
[1 0]


대단히 제한적인 데이터로부터 꽤 좋은 결과를 얻었다.   
그러나, 로지스틱 회귀는 데이터의 비선형 관계는 담아내지 못한다는 한계가 있다. 

이 결정 경계는 변수들 간의 비선형 관계를 가지고 있으므로 고급 분류 방법이 사용되어야 한다.  
K-평균, 결정 트리, 랜덤 포레스트 및 더 복잡한 신경망 등이 그것이다. 

### 더 큰 데이터셋의 분석  
사용할 데이터셋은 UCI에 공개되어 있는 와인 품질 데이터셋이다.  
- 독립변수(feature): 와인의 회분(ash), 알코올 농도(alcohol) 등 같은 여러 가지 화학적 속성들이다.  
- 종속변수: 전문가가 판정한 와인의 등급 

In [27]:
import pandas as pd

# csv 파일을 읽고 레코드들을 표시하기 
features = pd.read_csv('../data/winequality-red.csv')
features.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


우선 features 데이터프레임을 x와 y 프레임으로 분리한다.  
그런 다음 다시 훈련 프레임과 테스트 프레임으로 나눈다.  
훈련과 테스트 프레임의 비율이 80과 20이 되도록 무작위로 섞어 분리한다. 

In [31]:
x = features # all features
x.drop(['quality'], axis = 1) # y를 features로부터 분리한 것 
y = features[['quality']] # y만 features로 만든 것 # 리스트 두개 씌워야함 

print("x's features: ", x.columns)
print("y's columns: ", y.columns)

x's features:  Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')
y's columns:  Index(['quality'], dtype='object')


In [34]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2
)
print("Training_features: X", x_train.shape, y_train.shape)
print("Test features: y", x_test.shape, y_test.shape)

Training_features: X (1279, 12) (1279, 1)
Test features: y (320, 12) (320, 1)


### 정확도에 대한 측도: 정밀도 및 재현율  

다른 알고리즘과 비교하여 어떤 것이 더 정확한 지 비교하기 위해,  
하이퍼파라미터를 조정함으로써 예측 성능을 크게 개선할 수 있는데, 이 변화한 성능을 측정하기 위해  
정확도에 대한 측도가 필요하다. 

![nn](../screenshots/001.png)

정밀도(precision) =  
$$Precision = True Positive / (True Positive + False Positive)$$

재현율(Recall) =  
$$Recall = TruePositive / (TruePositive + FalseNegative)$$

우선, 로지스틱회귀를 이용해 와인 품질 데이터를 분류해 보자.  
와인의 종류별로 잘 구분되어 있으므로, 모델 평가를 위한 주 측도로서 정밀도를 사용한다.  

In [91]:
from sklearn.linear_model import LogisticRegression  
import numpy as np

# 모델 구축 
model = LogisticRegression()
# 훈련 데이터에 적합시키기 
model.fit(x_train, y_train)

# x_test에 대해서 y_test 예측하기
y_pred = model.predict(x_test)

print(y_pred.shape, type(y_pred)) # numpy 
print(y_test.shape, type(y_test)) # pandas

(320,) <class 'numpy.ndarray'>
(320, 1) <class 'pandas.core.frame.DataFrame'>


/Users/com/anaconda3/envs/env-keras/lib/python3.9/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/com/anaconda3/envs/env-keras/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [92]:
# y_test와 비교해서 정밀도 측정하기
# 공식대로 내가 직접 구현한 정밀도 
comparison = y_pred == np.array(y_test['quality'])
print(comparison.shape)

precision_own = np.count_nonzero(comparison == True) / len(y_pred)
print("own_정밀도: ", precision_own)

# 교재의 방식 
from sklearn.metrics import precision_score
# 예측에 대한 정밀도 계산 
print("Precision for Logistic Regression: ", precision_score(y_test, y_pred, average='micro')) 

(320,)
own_정밀도:  0.846875
Precision for Logistic Regression:  0.846875


이제 알고리즘을 더 추가해 모델을 만들어 보자.  

### K-최근접 이웃 모델(KNN - K-Nearest Neighbors)  
k개의 가장 가까운 이웃들을 기반으로 예측하도록 학습한다.  
새로운 점이 주어지면, 그 점과 가장 가까운 k개의 점들의 소속을 기반으로 새롤운 점의 소속 클래스 예측  


In [104]:
from sklearn.neighbors import KNeighborsClassifier

# KNN 모델 훈련 
model = KNeighborsClassifier(n_neighbors = 20)
model.fit(x_train, y_train)
# x_test에 대한 예측 
y_pred = model.predict(x_test)
# y_test와의 비교 
print("Precision for KNN: ", precision_score(y_test, y_pred, average='micro'))

Precision for KNN:  0.5375


/Users/com/anaconda3/envs/env-keras/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


knn 알고리즘은 훈련 데이터 전체를 보며, 각 신규 점에 대해서 가장 근접한 이웃 점들을 기반으로 점수를 낸다.  
KNN은 일반적으로 시간이 꽤 걸리며, 최고의 정확도가 나오지 않을 수도 있다.  
다른 알고리즘을 살펴보자. 

### 결정 트리 

결정 트리 전체를 구축하고 시각화할 수 있다.  
약간 복잡하지만 결정 트리를 시각화하고 싶다면 이 리스트처럼 하면 된다. 

In [105]:
from sklearn.tree import export_graphviz

# dot 파일 형식으로 저장 
export_graphviz(model, 
    out_file = '../screenshots/tree.dot', 
    feature_names=x_train.columns, 
    class_names=range(6), 
    rounded = True, proportion = False, 
    precision=1, filled=True
)

AttributeError: 'KNeighborsClassifier' object has no attribute 'tree_'

### 앙상블 기법 

많은 약분류기들의 예측을 결합해 강분류기를 구축하는 방법  
결정 트리에 앙상블 기법을 적용하면 랜덤 포레스트라고 하는 새로운 알록리즘이 탄생한다.  

랜덤 포레스트 :  
- 특성들의 부분집합과 데이터의 부분집합을 랜덤하게 선택한다.  
- 이 축소된 데이터를 가지고 결정 트리를 만든다.  

특성과 행들의 부분집합을로 여러 개의 결정 트리를 구축하고,  
그 결과들을 종합해 예측한다.  

회귀 모델을 구축하기 위해 랜덤 포레스트를 사용할 수도 있으며, 이 경우 각 트리의 평균값을 최종 답으로 출력한다. 

In [96]:
from sklearn.ensemble import RandomForestClassifier

# 100개의 랜덤 트리를 갖는 모델을 구축한다.  
model = RandomForestClassifier(n_estimators = 100)

# 훈련 데이터 사용 
model.fit(x_train, y_train)

# 테스트 데이터에 대한 예측 
y_pred = model.predict(x_test)

/Users/com/anaconda3/envs/env-keras/lib/python3.9/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [99]:
# 정밀도 출력 - 내가 직접 만든 정밀도
print(type(y_pred), y_pred.shape)
print(type(y_test), y_test.shape)

precision_own = np.count_nonzero(comparison == True) / len(y_pred)
print("own_정밀도: ", precision_own)

# 교재 - 정밀도 출력 
print("Precision for Random-Forest: ", precision_score(y_test, y_pred, average='micro'))

<class 'numpy.ndarray'> (320,)
<class 'pandas.core.frame.DataFrame'> (320, 1)
own_정밀도:  0.846875
Precision for Random-Forest:  0.978125


앙상블 기법을 사용해 훨씬 좋은 정밀도를 얻었다.  
앙상블 기법은 트리에만 국한되지 않으며, 다른 알고리즘들로도 결과물을 결합해  
분류기 대열을 만들 수 있다. 

### 편향 대 분산 : 미적합 대 과적합 

다트 보드에 다섯 개의 다트를 던진다고 생각해 보자.  

다섯 개의 다트가 좌측 상단에 잘 맞는 것은, 중심에는 잘 안 맞지만 아주 고르게 맞은 경우이다.  
이 경우는 큰 편향이 있는 경우이다. 

다섯 개의 다트가 보드 전체에 걸쳐 넓게 맞는 경우는 큰 분산이 존재하는 경우이다.  
이것이 큰 분산의 경우이다. 

편향과 분산을 조정해 타겟에 맞게 조준해야 한다. 

이제는 테스트 데이터에 대한 정밀도만 구하지 않고, 훈련 데이터와 테스트 데이터를 통한 정밀도 모두를 구할 것이다. 

In [100]:
from sklearn.linear_model import LogisticRegression  

# 로지스틱 회귀 모델의 구축 
model = LogisticRegression()

# 모델을 데이터에 맞춘다. 
model.fit(x_train, y_train)

# 훈련 데이터에 대한 예측 및 정밀도 
y_pred = model.predict(x_train)
print("Pricision for train data: ", precision_score(y_train, y_pred, average='micro'))

# 테스트 데이터에 대한 예측 및 정밀도
y_pred = model.predict(x_test)
print("Precision for test data: ", precision_score(y_pred, y_test, average='micro'))

Pricision for train data:  0.8592650508209538
Precision for test data:  0.846875


/Users/com/anaconda3/envs/env-keras/lib/python3.9/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/com/anaconda3/envs/env-keras/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


훈련 데이터에 대한 정밀도가 테스트 데이터에 대한 정밀도와 거의 동일하다.  
훈련 데이터에 더 잘 맞아야 한다. 이것은 미적합에 해당한다. 

미적합이란 훈련 데이터에서나 테스트 데이터에서 모두 잘 맞지 않은 경우이다. 

이것은 편향이라는 머신러닝 모델의 속성 때문에 발생한다.  
편향은 모델이 세우는 가정(assumption)에 해당하며, 편향이 크면 모델은 데이터로부터 잘 학습하지 못한다.  


모델에게 편향은 어느 정도 필요하다.  
그렇지 않으면 모델은 입력 데이터의 변동성에 대하여 매우 취약해지며 불량한 데이터가 들어오면 모델은 실수하게 될 것이다. 

In [107]:
from sklearn import tree
from sklearn.metrics import precision_score

model = tree.DecisionTreeClassifier()
# 모델을 데이터에 적합시키기
model.fit(x_train, y_train)
# 훈련 데이터에 대한 적합도 
y_pred = model.predict(x_train)
print("Precision for Train data: ", precision_score(y_train, y_pred, average='micro'))

# 테스트 데이터에 대한 적합도
y_pred = model.predict(x_test)
print("Precision for test data: ", precision_score(y_pred, y_test, average='micro'))
# 원래 0.6XXX가 나와야 한다. 

Precision for Train data:  1.0
Precision for test data:  1.0


훈련 데이터에 대한 정밀도는 100%인 반면, 테스트 데이터에 대한 정밀도는 떨어진다.  
이 모델은 훈련 데이터 패턴은 극도로 잘 학습하지 못헀다. 

이러한 모델은 큰 분산을 가졌다고 말하며, 훈련 데이터에 대하여 과적합하였다. 

우리는 테스트 데이터로서 제공하는 보지 못한 데이터에 대해서 모델이 잘 일반화하기를 원한다. 

머신러닝 모델의 분산은 입력 데이터의 변동에 따라 예측을 변경할 수 있는 능력을 결정한다.  
분산이 크다는 것은 입력 데이터에 적합하기 위해 모델이 예측을 계속 변경하기만 하고  
정말 패턴을 학습하지 못한다는 것을 의미한다. 

분산과 편향은 반비례한다. 편향을 크게 하면 분산이 작아지고 역도 성립한다.  
일반적으로 데이터 과학자는 편향과 분산 간의 절충을 받아들여야 한다. 

결정 트리와 랜덤 포레스트는 일반적으로 분산이 매우 크고 과적합하는 경향이 있다. 

일반적으로 현실 세계 데이터의 속성을 고려할 때, 편향이 너무 커지지 않으면서  
데이터의 모든 변동성을 포착하기 위해서는 대부분의 경우 비선형 모델이 필요하다. 

특성 수가 증가하고, 데이터셋이 더 복잡해지면, 특히 이미지, 텍스트, 오디오 등 비선형 데이터의 경우,  
데이터에 더 적합하고 모든 비선형성을 잡아낼 수 있는 더 복잡한 모델을 고려해야 한다.  
이것이 머신러닝에 속하는 딥러닝이라고 하는 또다른 커다란 분야의 시작이다. 